In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#import sys
#reload(sys)
#sys.setdefaultencoding('UTF8')

%matplotlib inline

# Navent Dataset
### Analisis de las postulaciones del dataset de Navent

In [ ]:
#cargo archivo csv de la educacion de los postulantes
p_educacion = pd.read_csv('../data/fiuba_1_postulantes_educacion.csv')
p_educacion.head()

In [ ]:
#cargo archivo csv del genero y edad de los postulantes
p_genero_edad = pd.read_csv('../data/fiuba_2_postulantes_genero_y_edad.csv')
p_genero_edad.head()

In [ ]:
#cargo archivo csv de las vistas
visitas = pd.read_csv('../data/fiuba_3_vistas.csv')
visitas.head()

In [ ]:
#cargo csv de postulaciones
postulaciones = pd.read_csv('../data/fiuba_4_postulaciones.csv')
postulaciones.head()

In [ ]:
#cargo csv de avisos
avisos_online = pd.read_csv('../data/fiuba_5_avisos_online.csv')
avisos_online.head()

In [ ]:
#cargo csv de detalle de avisos
avisos_detalle = pd.read_csv('../data/fiuba_6_avisos_detalle.csv')
avisos_detalle.head()

# Vista rapida de datos

In [ ]:
avisos_detalle.shape

In [ ]:
#columnas ciudad y mapacalle no deberian ser usadas, demasiados elementos null
avisos_detalle.isnull().sum()

In [ ]:
avisos_detalle.info()

In [ ]:
#dropeo las columnas con muchos null
avisos_detalle.drop('mapacalle', axis=1, inplace=True)
avisos_detalle.drop('ciudad', axis=1, inplace=True)

In [ ]:
postulaciones.shape

In [ ]:
postulaciones.isnull().sum()

In [ ]:
postulaciones.info()

In [ ]:
visitas.shape

In [ ]:
visitas.isnull().sum()

In [ ]:
visitas.info()

In [ ]:
p_educacion.shape

In [ ]:
p_educacion.isnull().sum()

In [ ]:
p_educacion.info()

In [ ]:
p_genero_edad.shape

In [ ]:
p_genero_edad.isnull().sum()

In [ ]:
p_genero_edad.info()

# Limpieza de datos
En esta seccion limpiaremos los datos y los dejaremos preparados para el análisis. Los pasos a seguir son:
- Convertir las fechas de postulaciones a datetime
- Limpiar los espacios en blanco y Nan de las fechas de nacimiento de los postulantes
- Los null de mapacalle y ciudad ???

In [ ]:
#Cambio la fecha a datetime
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'])
print(postulaciones['fechapostulacion'].max())
print(postulaciones['fechapostulacion'].min())

In [ ]:
postulaciones['fechapostulacion'].head()

In [ ]:
#Cambio timestamp a datetime
visitas['timestamp'] = pd.to_datetime(visitas['timestamp'])
print(visitas['timestamp'].max())
print(visitas['timestamp'].min())

In [ ]:
visitas['timestamp'].head()

In [ ]:
#vemos que hay fechas vacias
for x in p_genero_edad['fechanacimiento'].drop_duplicates():
    print ([x])

In [ ]:
#eliminamos los registros que tengan fecha de nacimiento nan
print(len(p_genero_edad))
p_genero_edad.dropna(inplace=True)
print(len(p_genero_edad))

In [ ]:
#siguen habiendo años invalidos
p_genero_edad.sort_values('fechanacimiento',ascending=False)

In [ ]:
#me quedo con los años validos
p_genero_edad = p_genero_edad[(p_genero_edad['fechanacimiento'].str.startswith('19'))| (p_genero_edad['fechanacimiento'].str.startswith('20'))]

In [ ]:
#veo que los años que empezaban en 00 no estan mas
p_genero_edad.sort_values('fechanacimiento',ascending=True).head()

In [ ]:
#calculo la edad de los postulantes
p_genero_edad['edad'] = pd.to_datetime('today').year-pd.to_datetime(p_genero_edad['fechanacimiento']).dt.year
p_genero_edad.head()

In [ ]:
gen_edad_validos=p_genero_edad.drop('fechanacimiento',axis=1)

In [ ]:
#aca me parece que habria que limpiar un poco los datos porque en educacion se repiten los id de postulante
#porque hay postulantes que aparecen varias veces con distintos estados de educacion
postulantes = pd.merge(p_educacion,gen_edad_validos,on="idpostulante",how="outer")
postulantes.head()

# Datos sobre niveles de educacion

In [ ]:
#cantidad de postulantes agrupado por nivel de educación
#usando el p_educacion estariamos tomando a cada postulante como varios si es que tiene mas de un estado de educacion
p_educacion.groupby('nombre').agg({'idpostulante':'count'}).sort_values('idpostulante',ascending=False)\
                                            .rename(columns={"idpostulante": "cantidad_postulantes"})

In [ ]:
p_educacion['estado'].value_counts()

In [ ]:
filtered = p_educacion.loc[(p_educacion.estado=='Graduado')]

fig = plt.figure()

ax = filtered['nombre'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Cuál es el máximo nivel educativo completado por los postulantes?', fontsize=16)

ax.set_xlabel('Nivel educativo', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
#¿Cuales son los niveles de estudio con mayor numero de estudiantes que se postulan?
filtered = p_educacion.loc[(p_educacion.estado=='En Curso')]

filtered['nombre'].value_counts()

In [ ]:
fig = plt.figure()

ax = filtered['nombre'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Cuales son los niveles de estudio con mayor numero de estudiantes que se postulan?', fontsize=16)

ax.set_xlabel('Nivel educativo', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
#este creo que tampoco es muy relevante

fig = plt.figure()

ax = p_educacion['estado'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('Estado del postulante', fontsize=16)

ax.set_xlabel('Estado del maximo nivel de estudio alcanzado', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

# Datos segun genero y edad

In [ ]:
gen_edad_validos['sexo'].value_counts()

In [ ]:
fig = plt.figure()

ax =gen_edad_validos['sexo'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('Cantidad de postulantes por genero', fontsize=16)

ax.set_xlabel('Genero', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
print(gen_edad_validos['edad'].max())
print(gen_edad_validos['edad'].min())

In [ ]:
#p_edad = p_edad[(p_edad['edad'] <= 65) & (p_edad['edad'] >= 18)]
#deje todas las edades pero se pueden reducir con la linea de arriba
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('¿Cuantas personas se postulan segun su edad?',  fontsize= 16)
ax.set_xlabel('Edad',  fontsize= 16)
ax.set_ylabel('Personas',  fontsize= 16)
gen_edad_validos['edad'].value_counts().sort_index().plot(kind='bar',figsize=(14,6))
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('¿Cuantas personas se postulan segun su edad?', fontsize= 16)
ax.set_xlabel('Edad', fontsize= 16)
ax.set_ylabel('Personas',fontsize= 16)
gen_edad_validos['edad'].plot(kind='hist',figsize=(13,6))
plt.show()

# Datos de avisos

In [ ]:
print(avisos_detalle['tipo_de_trabajo'].unique())

In [ ]:
#cantidad de avisos por tipo de trabajo (full time, part time...)
avisos_detalle.groupby('tipo_de_trabajo').agg({'tipo_de_trabajo':'count'}).sort_values('tipo_de_trabajo',ascending=False)\
                .rename(columns={"tipo_de_trabajo": "cantidad_avisos"})

In [ ]:
fig = plt.figure()

ax = avisos_detalle['tipo_de_trabajo'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Qué tipos de trabajo tienen mayor demanda?', fontsize=16)

ax.set_xlabel('Tipo de trabajo', fontsize= 16)
ax.set_ylabel('Cantidad de avisos', fontsize= 16);

In [ ]:
#empresas con mayor cantidad de avisos
avisos_detalle.groupby('denominacion_empresa').agg({'denominacion_empresa':'size'}).sort_values('denominacion_empresa',ascending=False)\
                      .rename(columns={"denominacion_empresa": "cantidad_avisos"}).head(20)      

In [ ]:
fig = plt.figure()

ax = avisos_detalle['denominacion_empresa'].value_counts().head(20).plot(kind='bar',figsize=(12,6));

plt.title('¿Qué empresas tienen mayor demanda de trabajo?', fontsize=16)

ax.set_xlabel('Empresas', fontsize= 16)
ax.set_ylabel('Cantidad de avisos', fontsize= 16);

In [ ]:
#Zonas con mayor cantidad de avisos
avisos_detalle.groupby('nombre_zona').agg({'nombre_zona':'size'}).sort_values('nombre_zona',ascending=False)\
                      .rename(columns={"nombre_zona": "cantidad_avisos"})   

In [ ]:
fig = plt.figure()

ax = avisos_detalle['nombre_zona'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿En qué zonas hay mayor demanda de trabajo? ', fontsize=16)

ax.set_xlabel('Zonas', fontsize= 16)
ax.set_ylabel('Cantidad de avisos', fontsize= 16);

In [ ]:
#Areas con mayor cantidad de avisos
avisos_detalle.groupby('nombre_area').agg({'nombre_area':'size'}).sort_values('nombre_area',ascending=False)\
                      .rename(columns={"nombre_area": "cantidad_avisos"}).head(10)

In [ ]:
fig = plt.figure()

ax = avisos_detalle['nombre_area'].value_counts().head(20).plot(kind='bar',figsize=(12,6));

plt.title('¿Qué áreas de trabajo tienen mayor demanda?', fontsize=16)

ax.set_xlabel('Area', fontsize= 16)
ax.set_ylabel('Cantidad de avisos', fontsize= 16);

In [ ]:
#Nivel laboral con mayor cantidad de avisos
avisos_detalle.groupby('nivel_laboral').agg({'nivel_laboral':'size'}).sort_values('nivel_laboral',ascending=False)\
                      .rename(columns={"nivel_laboral": "cantidad_avisos"})  

In [ ]:
fig = plt.figure()

ax = avisos_detalle['nivel_laboral'].value_counts().plot(kind='bar',figsize=(12,6));

plt.title('¿Qué niveles laborales tienen mayor demanda?', fontsize=16)

ax.set_xlabel('Nivel laboral', fontsize= 16)
ax.set_ylabel('Cantidad de avisos', fontsize= 16);

# Postulaciones por dia de semana 

In [ ]:
postulaciones['dia_semana'] = postulaciones['fechapostulacion'].dt.weekday_name

In [ ]:
postulaciones.head()

In [ ]:
postulaciones['dia_semana'].value_counts()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('¿Qué día de la semana se registran más postulaciones?', fontsize=16)
ax.set_xlabel('Dia de Semana', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
postulaciones['dia_semana'].value_counts().reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']).plot(kind='bar',figsize=(12,6))
plt.show()

# Postulaciones por dia del mes

In [ ]:
#Postulaciones enero
postulaciones_dia_mes = postulaciones[postulaciones['fechapostulacion'] < '02-01-2018']

In [ ]:
postulaciones_dia_mes.head()

In [ ]:
postulaciones_dia_mes['dia_mes'] = postulaciones_dia_mes.loc[:,'fechapostulacion'].dt.day

In [ ]:
postulaciones_dia_mes.head()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿Qué día del mes de Enero hubo más postulaciones?', fontsize=16)
ax.set_xlabel('Dia', fontsize=16)
ax.set_ylabel('Postulaciones',fontsize=16)
postulaciones_dia_mes['dia_mes'].value_counts().sort_index().plot(kind='bar',figsize=(12,6))

plt.show()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿Cómo se distribuyeron las postulaciones a lo largo de Enero?', fontsize=16)
ax.set_xlabel('Dia', fontsize=16)
ax.set_ylabel('Postulaciones',fontsize=16)
postulaciones_dia_mes['dia_mes'].value_counts().sort_index().plot(rot=90,linewidth=8,figsize=(11,6),alpha=0.75)

plt.show()

In [ ]:
#Postulaciones febrero
postulaciones_dia_mes = postulaciones[postulaciones['fechapostulacion'] >= '02-01-2018']

In [ ]:
postulaciones_dia_mes.head()

In [ ]:
postulaciones_dia_mes['dia_mes'] = postulaciones_dia_mes['fechapostulacion'].dt.day

In [ ]:
postulaciones_dia_mes.head()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿Qué día del mes de Febrero hubieron más postulaciones?', fontsize=16)
ax.set_xlabel('Dia', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
postulaciones_dia_mes['dia_mes'].value_counts().sort_index().plot(kind='bar', figsize=(11,6))

plt.show()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿Cómo se distribuyeron las postulaciones a lo largo de Febrero?', fontsize=16)
ax.set_xlabel('Dia', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
postulaciones_dia_mes['dia_mes'].value_counts().sort_index().plot(rot=90,linewidth=8,figsize=(11,6),alpha=0.75)

plt.show()

# Postulaciones por mes

In [ ]:
postulaciones['mes_postulacion']=postulaciones['fechapostulacion'].dt.month
postulaciones.head()

In [ ]:
postulaciones['mes_postulacion'].value_counts()

In [ ]:
import calendar
postulaciones['mes_postulacion'] = postulaciones['mes_postulacion'].apply(lambda x: calendar.month_abbr[x])

In [ ]:
postulaciones['mes_postulacion'].head()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿En qué mes hubieron más postulaciones?', fontsize=16)
ax.set_xlabel('Mes', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
postulaciones['mes_postulacion'].value_counts().sort_values().plot(kind='bar',figsize=(11,6))

plt.show()

# Postulaciones por franja horaria

In [ ]:
postulaciones['fechapostulacion'].dt.hour.value_counts()

In [ ]:
#capaz es mejor otro grafico para esto
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('Distribucion horaria de postulaciones', fontsize=16)
ax.set_xlabel('Hora', fontsize=16)
ax.set_ylabel('Hora', fontsize=16)
postulaciones['fechapostulacion'].dt.hour.plot(kind='hist',figsize=(11,6))

plt.show()

# Datos de vistas

In [ ]:
visitas.head()

In [ ]:
visitas['dia_semana'] = visitas['timestamp'].dt.weekday_name

In [ ]:
#capaz es mejor otro grafico para esto
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('¿En qué momento del día se registran más vistas de avisos?',fontsize=16)
ax.set_xlabel('Hora', fontsize=16)
ax.set_ylabel('Hora', fontsize=16)
visitas['timestamp'].dt.hour.plot(kind='hist',figsize=(11,6))

plt.show()

In [ ]:
#lo mismo que el anterior pero por cada hora
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿A qué hora del día se registran más vistas de avisos?', fontsize=16)
ax.set_xlabel('Hora', fontsize=16)
ax.set_ylabel('Visitas', fontsize=16)
visitas['timestamp'].dt.hour.value_counts().sort_index().plot(kind='bar', figsize=(11,6))

plt.show()

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿En qué momento del día se registran más vistas de avisos?', fontsize=16)
ax.set_xlabel('Hora', fontsize=16)
ax.set_ylabel('Visitas', fontsize=16)
visitas['timestamp'].dt.hour.value_counts().sort_index().plot(rot=90,linewidth=8,figsize=(11,6),alpha=0.75)

plt.show()

In [ ]:
print(visitas['timestamp'].min())
print(visitas['timestamp'].max())

In [ ]:
#agregar nombre de los dias de la semana
fig=plt.figure()
ax = fig.add_subplot(111)

plt.title('¿En qué dia de la semana se registran más vistas de avisos?', fontsize=16)
ax.set_xlabel('Dia', fontsize=16)
ax.set_ylabel('Visitas', fontsize=16)
visitas['dia_semana'].value_counts().reindex(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']).plot(kind='bar', figsize=(11,6))

plt.show()


# Analisis mas complejo de datos
Aca irian todos los analisis de los merge de los dataframes
Anoto un par de ejemplos aca :

- tipos de trabajo con mas postulantes por edad
- tipos de trabajo con mas postulantes graduados
- tipos de trabajo con mas postulantes estudiantes
- tipos de trabajo con mas postulantes estudiantes de secundario
- tipos de trabajo con mas postulantes estudiantes universitarios
- areas de trabajo con mas postulantes graduados
- areas de trabajo con mas postulantes estudiantes
- areas de trabajo con mas postulantes estudiantes universitarios


In [ ]:
#Top 10 avisos con mayor cantidad de postulantes
cantidad_postulaciones = postulaciones.groupby('idaviso').agg({'idpostulante':'count'}).sort_values('idpostulante',ascending = False).rename(columns={"idpostulante": "cantidad_postulantes"})
cantidad_postulaciones = cantidad_postulaciones.reset_index()
avisos_cantidad_postulantes = pd.merge(cantidad_postulaciones,avisos_detalle, on= 'idaviso', how='inner') 
avisos_cantidad_postulantes[['idaviso','cantidad_postulantes','titulo','descripcion']].sort_values('cantidad_postulantes',ascending= False).head(10)

In [ ]:
#Tipos de aviso con mayor cantidad de postulantes
avisos_cantidad_postulantes ['titulo'].value_counts().head(10)

In [ ]:
merge2 = pd.merge(postulaciones,avisos_detalle,on="idaviso",how="left")

In [ ]:
fig = plt.figure()

ax =merge2['nombre_zona'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Qué zona es la preferida por los postulantes?', fontsize=16)

ax.set_xlabel('Zona', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
fig = plt.figure()

ax =merge2['tipo_de_trabajo'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Qué tipo de trabajo es el más demandado por los postulantes?', fontsize=16)

ax.set_xlabel('Tipo de trabajo', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
fig = plt.figure()

ax =merge2['nivel_laboral'].value_counts().plot(kind='bar',figsize=(11,6));

plt.title('¿Qué nivel laboral es el más demandado por los postulantes?', fontsize=16)

ax.set_xlabel('Nivel laboral', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
fig = plt.figure()

ax = merge2['denominacion_empresa'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué empresas son las preferidas por los postulantes?', fontsize=16)

ax.set_xlabel('Empresas', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
postulantes_genero_avisos = pd.merge(merge2,gen_edad_validos,on="idpostulante",how="left")

In [ ]:
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='MASC')]

ax = filtered['denominacion_empresa'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué empresas son las más elegidas por los hombres?', fontsize=16)

ax.set_xlabel('Empresa', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes ', fontsize= 16);

In [ ]:
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='FEM')]

ax = filtered['denominacion_empresa'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué empresas son las más preferidas por las mujeres?', fontsize=16)

ax.set_xlabel('Empresa', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes ', fontsize= 16);

In [ ]:
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='MASC')]

ax = filtered['nombre_area'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué Areas de trabajo son las más elegidas por los hombres?', fontsize=16)

ax.set_xlabel('Area', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes ', fontsize= 16);

In [ ]:
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='FEM')]

ax = filtered['nombre_area'].value_counts().head(20).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué Areas de trabajo son las más elegidas por las mujeres?', fontsize=16)

ax.set_xlabel('Area', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes ', fontsize= 16);

In [ ]:
#Top 10  tipos de avisos con mayor cantidad de postulantes mujeres
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='FEM')]

ax = filtered['titulo'].value_counts().head(10).plot(kind='bar',figsize=(11,6));

plt.title('Top 10  tipos de avisos con mayor cantidad de postulantes mujeres', fontsize=24)

ax.set_xlabel('Avisos', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
fig = plt.figure()

filtered=postulantes_genero_avisos.loc[(postulantes_genero_avisos.sexo=='MASC')]

ax = filtered['titulo'].value_counts().head(10).plot(kind='bar',figsize=(11,6));

plt.title('Top 10  tipos de avisos con mayor cantidad de postulantes hombres', fontsize=16)

ax.set_xlabel('Avisos', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
filtered = postulantes_genero_avisos.loc[(postulantes_genero_avisos.tipo_de_trabajo=='Full-time')]

fig = plt.figure()

ax = filtered['edad'].value_counts().head(10).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué edad tienen la mayoría de los postulantes para trabajos Full-Time?', fontsize=16)

ax.set_xlabel('Edad', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

In [ ]:
filtered = postulantes_genero_avisos.loc[(postulantes_genero_avisos.tipo_de_trabajo=='Part-time')]

fig = plt.figure()

ax = filtered['edad'].value_counts().head(10).plot(kind='bar',figsize=(11,6));

plt.title('¿Qué edad tienen la mayoría de los postulantes para trabajos Part-Time?', fontsize=16)

ax.set_xlabel('Edad', fontsize= 16)
ax.set_ylabel('Cantidad de postulantes', fontsize= 16);

# Top 5 - Postulaciones x Area de trabajo

In [ ]:
df = pd.merge(postulaciones, avisos_detalle, on='idaviso', how='inner')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

plt.title('¿Cuales son las areas de trabajo más demandadas?', fontsize=16)
ax.set_xlabel('Area', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
df['nombre_area'].value_counts().head().plot(kind='bar', figsize=(11,6))

plt.show()

In [ ]:
#este me tira un memory error
fig = plt.figure()
ax = fig.add_subplot(111)

plt.title('Postulaciones x Zona (Dentro de ventas)', fontsize=16)
ax.set_xlabel('Zona', fontsize=16)
ax.set_ylabel('Postulaciones', fontsize=16)
merge2[merge2['nombre_area'] == 'Ventas']['nombre_zona'].value_counts().plot(kind='bar', figsize=(11,6))
plt.show()

# Tasa de conversion de avisos (postulaciones / visitas)

Agrego esto que puso natalia en Piazza para que lo lean. Por lo que vi, en el csv de vistas no hay idUsuario null, por lo que entiendo que en este csv solo tenemos las vistas de los usuarios que estaba logeados antes de postularse.
Entonces puede pasar que tengamos postulaciones de usuarios pero que no tengamos las vistas previas a la postulacion.
Tambien tenemos que chequear y limpiar que en las vistas seguramente hay usuarios duplicados.

Natalia Golmar 10 hours ago
Hola

Consulte este tema con la gente de Navent, y les comparto algunas respuestas:

- Las vistas se guardan solo para usuarios logueados en el sistema? No, se guardan todas las vistas y si el usuario no está logueado en el registro no tiene el campo idUsuario (no recuerdan si filtraron y nos enviaron sólo las vistas con usuarios, pero si es asi entiendan que seguro hay vistas sin idUsuario para los avisos)
- Es posible mirar avisos sin loguearse? Si
- Es posible que el usuario se loguee solo al postularse, y no quede registado el acceso al aviso como vista? Si
- Si un usuario visita mas de una vez un aviso, se registra cada vista o se registra solo la 1ra? Todas

In [ ]:
postulaciones.head()

In [ ]:
#me quedo con la fecha de la postulacion
postulaciones['fechapostulacion'] = postulaciones['fechapostulacion'].dt.date

In [ ]:
#lo mismo para el timestamp
visitas['timestamp'] = visitas['timestamp'].dt.date

In [ ]:
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(visitas['timestamp'])]

In [ ]:
visitas_validas = visitas[visitas['timestamp'].isin(postulaciones_validas['fechapostulacion'])]

In [ ]:
visitas_validas['timestamp'].value_counts()

In [ ]:
postulaciones_validas['fechapostulacion'].value_counts()

In [ ]:
postulaciones_validas = postulaciones_validas.rename(columns={'fechapostulacion':'fecha'})
postulaciones_validas.head()

In [ ]:
visitas_validas = visitas_validas.rename(columns={'idAviso':'idaviso','timestamp':'fecha'})
visitas_validas.head()

In [ ]:
vg = visitas_validas.groupby('idaviso').agg({'idpostulante':'count'})
vg = vg.reset_index()
pg = postulaciones_validas.groupby('idaviso').agg({'idpostulante':'count'})
pg = pg.reset_index()
tasa_convergencia = pd.merge(vg,pg,on='idaviso')

In [ ]:
tasa_convergencia.head()

In [ ]:
tasa_media = tasa_convergencia['idpostulante_x'].sum() / tasa_convergencia['idpostulante_x'].count()

In [ ]:
tasa_convergencia[tasa_convergencia['idpostulante_x'] > tasa_media].head()

In [ ]:
tasa_convergencia['tasa'] = (tasa_convergencia['idpostulante_x'] / tasa_convergencia['idpostulante_y'])

In [ ]:
tasa_convergencia.set_index('idaviso')['tasa'].sort_values(ascending=False).head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.title('Tasa de postulacion', fontsize=16)
ax.set_xlabel('Tasa', fontsize=16)
ax.set_ylabel('ID aviso', fontsize=16)
tasa_convergencia.set_index('idaviso')['tasa'].sort_values(ascending=False).head().plot(kind='bar', figsize=(11,6))
plt.show()

# Evolucion de cantidad de visitas por dia y el nivel de estudio de los postulantes

In [ ]:
df_merge = pd.merge(p_educacion,visitas,on='idpostulante')

In [ ]:
df_merge.groupby(['timestamp','estado']).agg({'idpostulante':'count'})

In [ ]:
heap = df_merge.pivot_table(index='nombre',columns='timestamp',values='idpostulante',aggfunc='count')

In [ ]:
heap

In [ ]:
plt.subplots(figsize=(15,10))
g = sns.heatmap(heap, cmap='YlGnBu')
plt.title('Evolucion de cantidad de visitas por dia y el nivel de estudio de los postulantes', fontsize=24)

# Heatmap educacion x estado

In [ ]:
heap = p_educacion.pivot_table(index='estado',columns='nombre',values='idpostulante',aggfunc='count')

In [ ]:
heap

In [ ]:
plt.subplots(figsize=(15,10))
g = sns.heatmap(heap)
plt.title('Cantidad de postulantes segun su educacion y estado', fontsize=24)